In [3]:
import gym
import numpy as np
from bento.client import Client
from bento.sim import Simulation
from bento.example.mountcar import MountainCar, Action, State
from bento.example.specs import Velocity, Position

In [ ]:
env = gym.make('MountainCar-v3')

In [ ]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [2]:
class MountainCarEnv:
    def __init__(self, host='bento.mrzzy.co', port='54242'):
        client = Client(host=host, port=port)
        client.connect(timeout_sec=30)
        if 'mountain_car' in client.list_sims():
            client.remove_sim('mountain_car')
        sim = Simulation.from_def(MountainCar, client)
        self.sim = sim
    def get_state(self):
        car = self.sim.entity(components=[Velocity, Position])
        return np.array([car[Position].x, car[Velocity].x])
    def reset(self):
        try: self.sim.stop()
        except: pass
        self.t = 0
        self.sim.start()
        return self.get_state()
    def step(self, action):
        env = self.sim.entity(components=[Action, State])
        env[Action].accelerate = action
        self.sim.step()
        self.t += 1
        state = self.get_state()
        reward = env[State].reward
        done = env[State].ended if self.t < 200 else True
        return state, reward, done